# Importacion de datos


In [ ]:
# data manipulation
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
from google.colab import files

# plot
import matplotlib.pyplot as plt

# machine learning frameworks
import tensorflow as tf
import torch
from torch.autograd import Variable
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_hub as hub


# python
import datetime
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

plt.style.use('tableau-colorblind10')
# plt.style.use('dark_background')
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score


importa datos

In [ ]:
social = pd.read_csv('/content/social.csv')
matriculaciones = pd.read_csv('/content/matriculaciones.csv')
academicos = pd.read_csv('/content/academicos.csv')
quiz = pd.read_csv('/content/quiz.csv')
entregas = pd.read_csv('/content/entregas.csv')
lecciones = pd.read_csv('/content/lecciones.csv')
foros = pd.read_csv('/content/foros.csv')
abandonos = pd.read_csv('/content/abandonos.csv')
abandonos_test = pd.read_csv('/content/abandonos_test.csv')

In [ ]:
variable = 'last' #con esta variable podemos cambiar si queremos los datos del ultimo año o del primero
#de todas maneras la mayorias de las variables que usamos son promedios de todos los años 

obtiene datos 

# Tratamiento de datos

In [ ]:
foros['año'] = foros['fecha'].apply(lambda x: pd.to_datetime(x).year)
participacion_en_foros = foros.groupby(['id','año'])['fecha'].count().reset_index(name='cantidad_foro')
cantidad_foro = participacion_en_foros.groupby(['id'])['cantidad_foro'].mean().reset_index(name='cantidad_foro')

In [ ]:
#entregas.calificacion[entregas['calficacion'] < 0] = 0
entregas.loc[(entregas['calficacion'] < 0) | (entregas['calficacion'].isnull()), 'calficacion'] = 0
quiz.loc[(quiz['grado'] < 0) | (quiz['grado'].isnull()), 'grado'] = 0

In [ ]:
entregas['año'] = entregas['fecha'].apply(lambda x: pd.to_datetime(x).year)
entregas_total = entregas.groupby(['id','año'])['fecha'].count().reset_index(name='cantidad')
entregas_total = entregas_total.groupby(['id'])['cantidad'].mean().reset_index(name='cantidad_entregas')

In [ ]:
entregas_calificacion = entregas.groupby(['id'])['calficacion'].mean().reset_index(name='calificacion')

In [ ]:
entregas_calificacion['calificacion'].fillna(value=0)
entregas_calificacion['calificacion'].astype(int)

In [ ]:
quiz['año'] = quiz['fecha_inicio'].apply(lambda x: pd.to_datetime(x).year)
participacion_quiz = quiz.groupby(['id','año'])['fecha_inicio'].count().reset_index(name='cantidad_quiz')
participacion_quiz = participacion_quiz.groupby(['id'])['cantidad_quiz'].mean().reset_index(name='quiz_entregas')
notas_quiz = quiz.groupby(['id'])['grado'].mean().reset_index(name='quiz_calificacion')

In [ ]:
notas_quiz['quiz_calificacion'].describe()

In [ ]:
# primero convertimos las fechas de string a un int que representa solo el año (prescindimos del mes/dia/hora/etc)
social['FS_fec'] = social['FS_fec'].apply(lambda x: pd.to_datetime(x).year)
social['SD_fn'] = social['SD_fn'].apply(lambda x: pd.to_datetime(x).year)
# decimos que la edad es la diferencia del año en que se registra el dato, y el año de nacimiento
social['edad'] = social['FS_fec'] - social['SD_fn']


## Cantidad de hijos

In [ ]:
social['SD_hijoQ'].fillna(0)
social['SD_hijoQ'] = social['SD_hijoQ'] / social['SD_hijoQ'].max()

In [ ]:
social['SD_hijoQ'].describe()

Fecha de nacimiento de los Hijos

In [ ]:
fig = plt.figure(figsize=(15, 5))

#hijos = ['HJ_fn_1', 'HJ_fn_2', 'HJ_fn_3']
#i = 0
#for h in hijos:
 # social[social[hijos[i]] > 0][hijos[i]].plot.hist(title='Distribución de Edades de Hijos', bins=10)
 # i=i+1

social[social['HJ_fn_1'] > 0]['HJ_fn_1'].plot.hist(title='Distribución de Edades de Hijos', bins=10)
social[social['HJ_fn_2'] > 0]['HJ_fn_2'].plot.hist(title='Distribución de Edades de Hijos', bins=10)
social[social['HJ_fn_3'] > 0]['HJ_fn_3'].plot.hist(title='Distribución de Edades de Hijos', bins=10)


Hay un estudiante que tiene un hijo nacido en 1920, eliminamos el registro

In [ ]:
social = pd.concat([social[social['HJ_fn_1'] > 1930], social[social['HJ_fn_1'].isna()]])
social = pd.concat([social[social['HJ_fn_2'] > 1930], social[social['HJ_fn_2'].isna()]])
social = pd.concat([social[social['HJ_fn_3'] > 1930], social[social['HJ_fn_3'].isna()]])
fig = plt.figure(figsize=(15, 5))
hijos = ['HJ_fn_1', 'HJ_fn_2', 'HJ_fn_3']
i = 0
for h in hijos:
  social[social[hijos[i]] > 0][hijos[i]].plot.hist(title='Distribución de Edades de Hijos', bins=10)
  i=i+1

Los normalizamos entre 0 y 1 tomando como año máximo 2019 y mínimo 1930

In [ ]:
social['HJ_fn_1'] = social['HJ_fn_1'].apply(lambda x: x/2019)
social['HJ_fn_2'] = social['HJ_fn_2'].apply(lambda x: x/2019)
social['HJ_fn_3'] = social['HJ_fn_3'].apply(lambda x: x/2019)

fig = plt.figure(figsize=(15, 5))

social[social['HJ_fn_1'] > 0]['HJ_fn_1'].plot.hist(title='Distribución de Edades de Hijos', bins=10)
social[social['HJ_fn_2'] > 0]['HJ_fn_2'].plot.hist(title='Distribución de Edades de Hijos', bins=10)
social[social['HJ_fn_3'] > 0]['HJ_fn_3'].plot.hist(title='Distribución de Edades de Hijos', bins=10)

## Fecha de nacimiento de los hijos

In [ ]:
academicos.loc[(academicos['inasistencia'] < 0) | (academicos['inasistencia'].isnull()), 'inasistencia'] = 0
academicos['año'] = academicos['periodo_id'].apply(lambda x: int(x.split('-')[0]))
academicos
calificaciones = academicos[academicos['calificacion'] >= 0]
calificaciones = calificaciones.groupby(['id'])['calificacion'].mean().reset_index(name='promedio')
inasistencias = academicos.groupby(['id'])['inasistencia'].sum().reset_index(name='inasistencia')


In [ ]:
calificaciones

In [ ]:
n = 0
mismo_departamento = pd.DataFrame(columns=('id', 'mismo_dep'))
for alumno in social['id'].unique().tolist():
   itr = social[social['id'] == alumno]   
   mismo_dep = ((itr['SD_ln_de'] == 'Durazno').any() & (itr['DP_itr'] == 'ITR Centro Sur').any()) or ((itr['SD_ln_de'] == 'Río Negro').any() & (itr['DP_itr'] == 'ITR Suroeste').any()) or ((itr['SD_ln_de'] == 'Rivera').any() & (itr['DP_itr'] == 'ITR Norte').any()) or ((itr['DP_ca'].str.contains('Licenciatura en Ciencia y Tecnología de Lácteos')).any() & (itr['SD_ln_de'] == 'Colonia').any()) or ((itr['DP_ca'].str.contains('Licenciatura en Análisis Alimentario')).any() & (itr['SD_ln_de'] == 'Paysandú').any()) or ((itr['DP_ca'].str.contains('Tecnólogo en Manejo de Sistemas')).any() & (itr['SD_ln_de'] == 'Colonia').any()) or ((itr['DP_ca'].str.contains('Licenciatura en Leche')).any() & (itr['SD_ln_de'] == 'Colonia').any()) or ((itr['DP_ca'].str.contains('Tecnólogo en Jazz')).any() & (itr['SD_ln_de'] == 'Florida').any())
   if mismo_dep:
    mismo_departamento.loc[n] = [alumno,1]
    n = n+1
   else:
    mismo_departamento.loc[n] = [alumno,0]
    n = n+1
#aqui creamos una variable binaria q seria si vive o no en el mismo departamento donde estudias


In [ ]:
n = 0 
materias_aprobacion = pd.DataFrame(columns=('id', 'materias_desaprobadas', 'materias_aprobadas'))
for alumno in academicos['id'].unique().tolist():
   calificacion = academicos[academicos['id']==alumno]   
   calificacion.dropna()
   calificacion_desaprobadas = calificacion[calificacion['calificacion']<3]
   calificacion_aprobadas = calificacion[calificacion['calificacion']>3]
   materias_aprobacion.loc[n] = [alumno, calificacion_desaprobadas['calificacion'].count(), calificacion_aprobadas['calificacion'].count()]
   n = n+1
#aqui estariamos contando cuantas materias aprobo o reprobo cada alumno 

In [ ]:
n = 0
weight = pd.DataFrame(columns=('id', 'weight'))
for alumno in abandonos['id'].unique().tolist():
   abandono_alumno = abandonos[abandonos['id'] == alumno]   
   if (abandono_alumno['abandono'] == 1).any():
    weight.loc[n] = [alumno,1.0]
    n = n+1
   else:
    weight.loc[n] = [alumno,0.2]
    n = n+1
#creamos esta columna para definir el "peso" de cada registro, como tenemos menos alumnos que dejaron, les damos una mayor relevancia 

In [ ]:
datos = matriculaciones

In [ ]:
datos = pd.merge(datos, social, on='id', how='left')
#merge es parecido a los join de las bases de datos, on indica la columna sobre la cual se los une
#el how seria como el tipo de join 

In [ ]:
datos = pd.merge(datos, entregas_total, on='id', how='left')

In [ ]:
datos = pd.merge(datos, participacion_en_foros, on='id', how='left')

In [ ]:
datos = pd.merge(datos, calificaciones, on='id', how='left')

In [ ]:
datos = pd.merge(datos, participacion_quiz, on='id', how='left')

In [ ]:
datos = pd.merge(datos, mismo_departamento, on='id', how='left')

In [ ]:
datos = pd.merge(datos, entregas_calificacion, on='id', how='left')

In [ ]:
datos = pd.merge(datos, notas_quiz, on='id', how='left')
datos = pd.merge(datos, inasistencias, on='id', how='left')
datos = pd.merge(datos, weight, on='id', how='left')
datos = pd.merge(datos, materias_aprobacion, on='id', how='left')


In [ ]:
datos

In [ ]:
datos.edad = datos.edad.fillna(value=0)#fillna completa los valores nulos para evitar errores
datos.edad = datos.edad.astype(int)
datos.promedio = datos.promedio.fillna(value=0)
datos.promedio = datos.promedio.astype(int)
datos.DP_ca = datos.DP_ca.fillna(value='Desconocido')
datos.DP_ca = datos.DP_ca.astype(str)
datos.SD_sex = datos.SD_sex.fillna(value='Desconocido')
datos.SD_re_de = datos.SD_re_de.fillna(value='Desconocido')
datos.SD_re_de = datos.SD_re_de.astype(str)
datos.SD_sex = datos.SD_sex.astype(str)
datos.DP_itr = datos.DP_itr.fillna(value='Desconocido')
datos.DP_itr = datos.DP_itr.astype(str)
datos.ED_ms_o = datos.ED_ms_o.fillna(value='Desconocido')
datos.ED_ms_o = datos.ED_ms_o.astype(str)
datos.ED_rep = datos.ED_rep.fillna(value='Desconocido')
datos.ED_rep = datos.ED_rep.astype(str)
datos.SD_fi_ne = datos.SD_fi_ne.fillna(value='Desconocido')
datos.SD_fi_ne = datos.SD_fi_ne.astype(str)
datos.cantidad_entregas = datos.cantidad_entregas.fillna(value='0')
datos.cantidad_entregas = datos.cantidad_entregas.astype(int)
datos.quiz_entregas = datos.quiz_entregas.fillna(value=0)
datos.quiz_entregas = datos.quiz_entregas.astype(int)
datos.SD_fi_tt = datos.SD_fi_tt.fillna(value=0)
datos.SD_fi_tt = datos.SD_fi_tt.astype(int)
datos.SD_fi_th = datos.SD_fi_th.fillna(value=0)
datos.SD_fi_th = datos.SD_fi_th.astype(int)
datos.EP_m = datos.EP_m.fillna(value='Desconocido')
datos.EP_m = datos.EP_m.astype(str)
datos.EP_p = datos.EP_p.fillna(value='Desconocido')
datos.EP_p = datos.EP_p.astype(str)
datos.TR_ct = datos.TR_ct.fillna(value='Desconocido')
datos.TR_ct = datos.TR_ct.astype(str)
datos.cantidad_foro = datos.cantidad_foro.fillna(value=0)
datos.cantidad_foro = datos.cantidad_foro.astype(int)
datos.VH_to = datos.VH_to.fillna(value='Desconocido')
datos.VH_to = datos.VH_to.astype(str)
datos.SD_hijos = datos.SD_hijos.fillna(value=0)
datos.SD_hijos = datos.SD_hijos.astype(int)
datos.ED_auaa = datos.ED_auaa.fillna(value=0)
datos.ED_auaa = datos.ED_auaa.astype(int)
datos.mismo_dep = datos.mismo_dep.fillna(value=0)
datos.mismo_dep = datos.mismo_dep.astype(int)
datos.calificacion = datos.calificacion.fillna(value=0)
datos.calificacion = datos.calificacion.astype(int)
datos.quiz_calificacion = datos.quiz_calificacion.fillna(value=0)
datos.quiz_calificacion = datos.quiz_calificacion.astype(int)
datos.inasistencia = datos.inasistencia.fillna(value=0)
datos.inasistencia = datos.inasistencia.astype(int)
datos.plan_id = datos.plan_id.fillna(value='Desconocido')
datos.plan_id = datos.plan_id.astype(str)
datos.TR_rtc = datos.TR_rtc.fillna(value=0)
datos.TR_rtc = datos.TR_rtc.astype(int)
datos.materias_desaprobadas = datos.materias_desaprobadas.fillna(value=0)
datos.materias_desaprobadas = datos.materias_desaprobadas.astype(int)
datos.materias_aprobadas = datos.materias_aprobadas.fillna(value=0)
datos.materias_aprobadas = datos.materias_aprobadas.astype(int)

In [ ]:
datos.SD_hijoQ = datos.SD_hijoQ.fillna(value=0)
datos.SD_hijoQ = datos.SD_hijoQ.astype(int)


In [ ]:
datos.SD_hijoQ

In [ ]:
datos['plan_id'].unique()

In [ ]:
datos_test = pd.merge(abandonos_test, datos, on='id', how='left')
datos = pd.merge(abandonos, datos, on='id', how='left')

merge convina los datos, parecido a un join de base de datos de ahi el left/rigth/inner etc 

elimnamos datos nulos y unificamos el tipo de datos para evitar errores

In [ ]:
datos_test = datos_test.drop_duplicates(keep=variable, subset='id')

In [ ]:
datos = datos.drop_duplicates(keep=variable, subset='id')

In [ ]:
entregas_calificacion

In [ ]:
social['SD_ln_de'].unique()

In [ ]:
social['DP_itr'].unique()

elimino los datos de otros años y me quedo con los del ultimo o primer año para simplificar, cambiando last por first o viceversa 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit_transform(datos.promedio.values.reshape(-1, 1))
scaler.fit_transform(datos.inasistencia.values.reshape(-1, 1))
scaler.fit_transform(datos.materias_desaprobadas.values.reshape(-1, 1))
scaler.fit_transform(datos.materias_aprobadas.values.reshape(-1, 1))
scaler.fit_transform(datos.quiz_calificacion.values.reshape(-1, 1))
scaler.fit_transform(datos.quiz_entregas.values.reshape(-1, 1))
scaler.fit_transform(datos.edad.values.reshape(-1, 1))
scaler.fit_transform(datos.cantidad_foro.values.reshape(-1, 1))
scaler.fit_transform(datos.cantidad_entregas.values.reshape(-1, 1))
scaler.fit_transform(datos.TR_rtc.values.reshape(-1, 1))
scaler.fit_transform(datos.SD_hijoQ.values.reshape(-1, 1))
scaler.fit_transform(datos.ED_auaa.values.reshape(-1, 1))
scaler.fit_transform(datos.SD_fi_th.values.reshape(-1, 1))
scaler.fit_transform(datos.SD_fi_tt.values.reshape(-1, 1))

In [ ]:


columnas_datos = ['weight','materias_aprobadas', 'materias_desaprobadas','inasistencia','TR_rtc','plan_id','calificacion','quiz_calificacion','mismo_dep','SD_hijos','SD_hijoQ','ED_auaa','SD_fi_ne','id','SD_re_de', 'SD_sex', 'VH_to','edad','promedio','DP_itr','ED_ms_o','ED_rep','DP_ca', 'quiz_entregas','SD_fi_tt', 'SD_fi_th','EP_m','EP_p','TR_ct', 'cantidad_foro', 'cantidad_entregas']
x = datos[columnas_datos]
y = datos['abandono']
x_prueba = datos_test[columnas_datos]


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

crea las datos x e y

In [ ]:
datos_abandono = datos[datos['abandono']==1]
datos_no_abandono = datos[datos['abandono'] ==0]

creo dos dataset, uno con todos los que abandonaron y otro con los que no para comparar datos en busqueda de buenas variables

---



In [ ]:
#datos_abandono['SD_fi_ne'].describe()
datos_abandono['TR_rtc'].value_counts().plot(kind='bar')


In [ ]:
datos['TR_rtc'].unique()


In [ ]:
#datos_no_abandono['SD_fi_ne'].describe()
datos_no_abandono['TR_rtc'].value_counts().plot(kind='bar')



In [ ]:
datos_abandono

In [ ]:
x_train

# Creacion de columnas tf

In [ ]:
#el modelo dnn usa este tipo de columnas

sex = tf.feature_column.categorical_column_with_vocabulary_list('SD_sex',['f, m, Desconocido'])#este tipo de columna es para categorias con pocas opciones
SD_sex = tf.feature_column.indicator_column(sex)#la convertimos en columna indicafor para que lo entienda tensorflow
re_de = tf.feature_column.categorical_column_with_hash_bucket('SD_re_de', hash_bucket_size=1000)#este tipo de columna es para categorias con muchas opciones
SD_re_de = tf.feature_column.embedding_column(re_de, dimension=19)#tambien hay que transformarlo a este tipo para q lo entienda tensorflow estimator
edad = tf.feature_column.numeric_column('edad')#tipo de columna numerica
promedio = tf.feature_column.numeric_column('promedio')
DP_c = tf.feature_column.categorical_column_with_hash_bucket('DP_ca', hash_bucket_size=1000)
DP_ca = tf.feature_column.embedding_column(DP_c, dimension=24)
DP = tf.feature_column.categorical_column_with_vocabulary_list('DP_itr',['ITR Norte', 'ITR Suroeste', 'Desconocido', 'ITR Centro Sur'])
DP_itr = tf.feature_column.indicator_column(DP)
ED_ms = tf.feature_column.categorical_column_with_vocabulary_list('ED_ms_o', ['centifica', 'humanistica', 'bachillerato-tecnologico',
       'Desconocido', 'biologica', 'Seleccione una opción',
       'cursos-tecnicos-nivel-medio', 'arte-expresion'])
ED_ms_o = tf.feature_column.indicator_column(ED_ms)
rep = tf.feature_column.categorical_column_with_hash_bucket('ED_rep', hash_bucket_size=1000)
ED_rep = tf.feature_column.embedding_column(rep, dimension=5)
cantidad_entregas = tf.feature_column.numeric_column('cantidad_entregas')
quiz_entregas = tf.feature_column.numeric_column('quiz_entregas')
SD_fi_tt = tf.feature_column.numeric_column('SD_fi_tt')
SD_fi_th = tf.feature_column.numeric_column('SD_fi_th')
EPp = tf.feature_column.categorical_column_with_hash_bucket('EP_p', hash_bucket_size=1000)
EP_p = tf.feature_column.embedding_column(EPp, dimension=11)
EPm = tf.feature_column.categorical_column_with_hash_bucket('EP_m', hash_bucket_size=1000)
EP_m = tf.feature_column.embedding_column(EPm, dimension=11)
TR = tf.feature_column.categorical_column_with_hash_bucket('TR_ct', hash_bucket_size=1000)
TR_ct = tf.feature_column.embedding_column(TR, dimension=10)
cantidad_foro = tf.feature_column.numeric_column('cantidad_foro')
VH = tf.feature_column.categorical_column_with_hash_bucket('VH_to', hash_bucket_size=1000)
VH_to = tf.feature_column.embedding_column(VH, dimension=8)
SD_hijos = tf.feature_column.numeric_column('SD_hijos')
SD_hijoQ = tf.feature_column.numeric_column('SD_hijoQ')
ED_auaa = tf.feature_column.numeric_column('ED_auaa')
SD_fi = tf.feature_column.categorical_column_with_vocabulary_list('SD_fi_ne', ['posgrado_completo_o_incompleto', 'terciarios_completos',
       'secundaria_completa_o_incompleta', 'Desconocido',
       'terciarios_incompletos', 'primaria_o_menos'])
SD_fi_ne = tf.feature_column.indicator_column(SD_fi, )
mismo_dep = tf.feature_column.numeric_column('mismo_dep')
quiz_calificacion = tf.feature_column.numeric_column('quiz_calificacion')
entregas_calificacion = tf.feature_column.numeric_column('calificacion')
inasistencias = tf.feature_column.numeric_column('inasistencia')
plan = tf.feature_column.categorical_column_with_hash_bucket('plan_id', hash_bucket_size=1000)
plan_id = tf.feature_column.embedding_column(plan, dimension=30)

TR_rtc = tf.feature_column.numeric_column('TR_rtc')
weight = tf.feature_column.numeric_column('weight')
materias_aprobadas = tf.feature_column.numeric_column('materias_aprobadas')
materias_desaprobadas = tf.feature_column.numeric_column('materias_desaprobadas')
 

In [ ]:
f =tf.estimator.BaselineClassifier(n_classes=3)

crea los tensores de tensorflow q seria como sus columnas, sequence column seria para los valores string tipo departamento, y numeric para valores numericos, categorical para cuando son pocos valores y ya los sabes

# Columnas a usar

In [ ]:
#aqui van las columnas que efectivamente se van a usar en el algoritmo 
columnas = [weight,materias_aprobadas,materias_desaprobadas,ED_ms_o,inasistencias,TR_rtc,plan_id,mismo_dep,quiz_entregas,promedio,entregas_calificacion, edad,SD_fi_ne, SD_hijos, SD_hijoQ, DP_ca, EP_m,SD_sex,SD_re_de, ED_rep, DP_itr,cantidad_entregas ]

#aqui son las mismas columnas pero quito la columna weight
columnas_para_clasificacion = [inasistencias,materias_aprobadas,materias_desaprobadas,TR_rtc,plan_id,mismo_dep,quiz_entregas,promedio,entregas_calificacion, edad,SD_fi_ne, SD_hijos, SD_hijoQ, DP_ca, EP_m,SD_sex,SD_re_de, ED_rep, DP_itr,cantidad_entregas ]



lista las columnas

In [ ]:
funcion_entrada = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)

rellena los tensores

# Creacion de algoritmo

In [ ]:
'''modelo = tf.estimator.LinearClassifier(feature_columns=columnas,optimizer='Adagrad')
  #algoritmo linear
 
modelo = tf.estimator.DNNClassifier(
 feature_columns=columnas_sin_weight, #modelo deep neural network
 hidden_units=[1024,254,36],#estos numeros son la cantidad de 'neuronas' por capa, por ejemplo 3 capas de 60/10/2 etc                                     
 optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.01,
                  #          l1_regularization_strength=0.001,
                              #                          l2_regularization_strength=0.001

),
 n_classes=2,#numero de clasificaciones a predecir
 dropout=0.1,
 model_dir='./tmp/mnist_model',
 #optimizer=tf.train.AdamOptimizer(1e-4,),#optimizador del algoritmo y algoritmo, hay varios a elegir en la documentacion
 weight_column=weight,
 )
'''


#este ultimo seria como una convinacion de ambos algoritmos linear y dnn, de todas maneras no obtuvimos mejoras grandes
modelo = tf.estimator.DNNLinearCombinedClassifier(
    linear_feature_columns=columnas_para_clasificacion,
    linear_optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.01),
    dnn_feature_columns=columnas_para_clasificacion,
    dnn_hidden_units=[1024,252,42],
    dnn_dropout=0.1,
    dnn_optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.01),
    weight_column = weight
   )





selecciona el algoritmo a usar

# Entrenamiento del algoritmo 

In [ ]:
modelo.train(input_fn=funcion_entrada, steps=8000)

In [ ]:
modelo.evaluate

entrena el algoritmo, mientras menor sea el 'loss' mejor esta haciendo el sistema. Steps es la cantidad de veces que se entrena, mas steps mejora el sistema

In [ ]:
funcion_prediccion = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test), shuffle=False)
funcion_prediccion_prueba = tf.estimator.inputs.pandas_input_fn(x=x_prueba,batch_size=len(x_prueba), shuffle=False)


hace la prediccion

In [ ]:
generador_predicciones = modelo.predict(input_fn=funcion_prediccion)
#aqui estamos prediciendo sobre los datos de entrenamiento

generador_predicciones_prueba = modelo.predict(input_fn=funcion_prediccion_prueba)
#aqui sobre los datos para entregar

In [ ]:
generador_predicciones_prueba

pasa la prediccion a una lista para luego separar las predicciones etc 





In [ ]:
predicciones = list(generador_predicciones)
predicciones_prueba = list(generador_predicciones_prueba)

In [ ]:
predicciones_finales = [prediccion['class_ids'][0] for prediccion in predicciones]

#la prediccion devuelve objetos en donde el atributo class_ids seria el valor predicho

In [ ]:
predicciones_finales_prueba = [prediccion['class_ids'][0] for prediccion in predicciones_prueba]

#las clases terminadas en _prueba serian las de el abandonos test que son para entregar

toma el valor 'y' de casa prediccion

In [ ]:
from sklearn.metrics import classification_report

# Resultados

In [ ]:
print(classification_report(y_test,predicciones_finales))

diferentes metricas sobre la presicion

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
matriz = confusion_matrix(y_test, predicciones_finales)
print(matriz)

imprime resultados valores 1 acertados sobre uno errados y luego 

---

valores 0 errados sobre 0 acertados

---



In [ ]:
roc_auc = roc_auc_score(y_test, predicciones_finales)
print(roc_auc)

es un valor del 0 al 1, mientras mas cerca del 1 mas exacto es el modelo de prediccines

In [ ]:
datos_prediccion = pd.DataFrame(columns=['id', 'abandono'] )
datos_prediccion_prueba = pd.DataFrame(columns=['id', 'abandono'] )

#aqui se genera datasets que completaremos con datos para hacer el cvs

In [ ]:
x=0
for i in x_test['id']:
  datos_prediccion.loc[x] = [i,predicciones_finales[x]]
  x = x+1 
  
#completamos el dataframe  

In [ ]:
x=0
for i in x_prueba['id']:
  datos_prediccion_prueba.loc[x] = [i,predicciones_finales_prueba[x]]
  x = x+1
  
#completamos el dataframe con los datos para el cvs

In [ ]:
archivo_resultados = datos_prediccion_prueba.to_csv('resultados.csv')
#generamos el cvs

In [ ]:
#files.download('resultados.csv')
#descargamos el cvs, ejecutar por separado